# Timeline Visualisation Creation
The purpose of this Jupyter Notebook is to create an interactive timeline using Plotly which depicts the key events before, during and after the Gamestop short squeeze.

In [1]:
import os
import json
import requests

import pandas as pd
import plotly.graph_objects as go

Loading pickled dataframe

In [2]:
pickle_file_path = '../scraping/gme_daily_transformed_df.pkl'
gme_daily_transformed_df = pd.read_pickle(pickle_file_path)

# Filtering the DataFrame to include only the data from January 2021 to April 2021
gme_jan_apr2021_df = gme_daily_transformed_df[(gme_daily_transformed_df['Date'] >= '2021-01-01') & (gme_daily_transformed_df['Date'] <= '2021-04-30')].copy()

Creating a new column in the dataframe called 'Event' to store the key events which occured throughout the period

In [3]:
gme_jan_apr2021_df['Event'] = pd.NA

gme_jan_apr2021_df.reset_index(inplace=True)
gme_jan_apr2021_df['Date'] = pd.to_datetime(gme_jan_apr2021_df['Date'])

# Adding the events to the DataFrame based on the dates
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-01-04'), 'Event'] = 'First Trading Day of 2021'
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-01-11'), 'Event'] = 'New Leaders Appointed'
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-01-22'), 'Event'] = 'Shares Surge by 50%'
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-01-27'), 'Event'] = 'Highest Close'
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-01-28'), 'Event'] = 'Robinhood Restriction'
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-02-04'), 'Event'] = 'Restrictions Lifted'
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-02-19'), 'Event'] = 'Post Squeeze Low'
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-02-24'), 'Event'] = 'Congress Hearing'
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-03-10'), 'Event'] = 'Post Squeeze High'

### Plotting the timeline using Plotly

In [4]:
df = gme_jan_apr2021_df

fig = go.Figure()

# Adding line for stock price
fig.add_trace(go.Scatter(
    x=df['Date'], 
    y=df['Close'], 
    mode='lines+markers',
    name='Stock Price',
    marker=dict(size=6, color='blue'),
    hovertemplate='%{x}: $%{y:.2f}<extra></extra>'
))

# Offseting to avoid overlap
y_offset = (max(df['Close']) - min(df['Close'])) * 0.05

annotations = []
# Start with the first event below the line and alternate above/below for each subsequent event
above = False

for index, row in df.iterrows():
    if pd.notnull(row['Event']):
        triangle_y = row['Close'] + y_offset if above else row['Close'] - y_offset
        
        # Adding red dot for the event
        fig.add_trace(go.Scatter(
            x=[row['Date']],
            y=[row['Close']],
            mode='markers',
            marker=dict(size=12, color='red'), 
            hoverinfo='skip',  
            showlegend=False
        ))
        
        # Adding a red triangle for the event
        fig.add_trace(go.Scatter(
            x=[row['Date']],
            y=[triangle_y], 
            mode='markers',
            marker=dict(
                symbol='triangle-down' if above else 'triangle-up',
                size=16,
                color='red'
            ),
            hoverinfo='skip',
            showlegend=False
        ))
        
        text_y = triangle_y + (y_offset / 2 if above else -y_offset / 2)
        
        # Formatting the annotations
        annotations.append(dict(
            x=row['Date'],
            y=text_y,
            xref='x',
            yref='y',
            text=row['Event'], 
            showarrow=False,
            font=dict(size=10, color='red'),
            bgcolor='rgba(255,255,255,0.9)',
            borderpad=4,
            xanchor='center',
            yanchor='bottom' if above else 'top'
        ))
        
        above = not above

fig.update_layout(
    title={
        'text': 'GameStop Stock Surge Timeline',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_font=dict(size=24),
    xaxis_title='Date',
    yaxis_title='Stock Price ($)',
    xaxis=dict(
        rangeslider=dict(visible=True), 
        type='date'
    ),
    hovermode='closest',
    annotations=annotations
)

# Saving the plot to the visualisations and docs folders
fig.write_html("interactive_timeline.html")
fig.write_html("../docs/interactive_timeline.html")
